In [208]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
from scipy.special import expit

In [209]:
def abrirImagenesEscaladas( carpeta, escala=250 ):
    # abre todas las imagenes de la carpeta, y las escala de tal forma que midan (escala x escala)px
    # devuelve las imagenes aplanadas -> vectores de tamano escala^2 con valores entre 0 y 1
    imagenes = []

    for dirpath, dirnames, filenames in os.walk(carpeta):
        for file in filenames:
            img = Image.open( os.path.join(carpeta, file) )
            img = img.resize((escala, escala))
            img.convert('1')
            img = np.asarray(img)
            if len(img.shape)==3:
                img = img[:,:,0].reshape((escala**2 )) / 255
            else:
                img = img.reshape((escala**2 )) / 255
            
            imagenes.append( img )

    return imagenes

In [210]:
def balancear_datos(imagenes_entrenamiento):
    imagenes_entrenamiento_balanceadas = abrirImagenesEscaladas(imagenes_entrenamiento)
    return imagenes_entrenamiento_balanceadas

In [211]:
imagenes_sanos = np.array(balancear_datos("train_mini/sano")) #len = 349
imagenes_neum = np.array(balancear_datos("train_mini/neumonia")) #len = 349


In [212]:
# Definir el número de iteraciones y la tasa de aprendizaje
MAX_ITER = 1000
TOLERANCIA = 0.0001

In [213]:
def f(x,w,b):
    return (np.tanh(np.dot(x,w)+b)+1)/2

In [214]:
def L(x,d,w,b):
    y_p = f(x,w,b)
    return (y_p-d)**2

In [215]:
# Derivadas de L(w, b)
def dW(x,d,w,b):
    y_p = f(x,w,b)
    return (y_p-d)*(1-np.tanh(np.dot(x,w)+b)**2) * x

def dB(x,d,w,b):
    y_p = f(x,w,b)
    return (y_p-d)*(1-np.tanh(np.dot(x,w)+b)**2)

In [216]:
# Función de búsqueda del mínimo de L(w, b)
def gradiente_descendente(x_sanos,x_enfermos,alpha):
    # Valores iniciales de w y b
    w_t = np.zeros(x_sanos.shape[1]) #x_sanos y x_enfermos del mismo tamaño
    b_t = 0

    # Mientras no hayamos llegado al maximo de iteraciones
    iter = 0
    error_anterior = float('inf')

    while iter <= MAX_ITER:
        # Computamos siguientes w y b a partir de las derivadas de la función
        grad_w = 0
        grad_b = 0
        error = 0

        
        d_sanos=0 #diagnostico para el conjunto de imagenes sin neumonía
        for i in range(len(x_sanos)):  
            grad_w += dW(x_sanos[i],d_sanos,w_t,b_t)
            grad_b += dB(x_sanos[i],d_sanos,w_t,b_t)
            error += L(x_sanos[i],d_sanos,w_t,b_t)
        
        d_neum=1 #diagnostico para el conjunto de imagenes con neumonía
        for i in range(len(x_enfermos)):
            grad_w += dW(x_enfermos[i],d_neum,w_t,b_t)
            grad_b += dB(x_enfermos[i],d_neum,w_t,b_t)
            error+=L(x_enfermos[i],d_neum,w_t,b_t)

        #se toma promedio porque de otra manera los numeros dan muy grandes y se entorpece la solucion
        grad_w /= (len(x_sanos) + len(x_enfermos))
        grad_b /= (len(x_sanos) + len(x_enfermos))
        error /= (len(x_sanos) + len(x_enfermos))
        
        print(f'iteracion: {iter}, error: {error}')

        w_tsig = w_t - alpha * grad_w
        b_tsig = b_t - alpha * grad_b

        # Chequeamos si ya alcanzamos la convergencia
        if abs(error_anterior - error) < TOLERANCIA:
            break

        # Preparamos la siguiente iteración
        w_t = w_tsig
        b_t = b_tsig
        error_anterior = error
        iter = iter + 1

    return w_t, b_t

In [217]:
alfa = 0.001
w,b = gradiente_descendente(imagenes_sanos,imagenes_neum,alfa)

iteracion: 0, error: 0.25
iteracion: 1, error: 0.23575982338326107
iteracion: 2, error: 0.3494242965893193
iteracion: 3, error: 0.48879288621947614
iteracion: 4, error: 0.4809378075024566
iteracion: 5, error: 0.45287715727578715
iteracion: 6, error: 0.23746607884299617
iteracion: 7, error: 0.4896723043068773
iteracion: 8, error: 0.48103918316147715
iteracion: 9, error: 0.44475257021384684
iteracion: 10, error: 0.18831700643787022
iteracion: 11, error: 0.2090957080052777
iteracion: 12, error: 0.4857656368438576
iteracion: 13, error: 0.4721412540889991
iteracion: 14, error: 0.4005394906948017
iteracion: 15, error: 0.3805457600959638
iteracion: 16, error: 0.45298920167257695
iteracion: 17, error: 0.22669556533060709
iteracion: 18, error: 0.49600541381718877
iteracion: 19, error: 0.4949491919906848
iteracion: 20, error: 0.4932058904933627
iteracion: 21, error: 0.48988680854407
iteracion: 22, error: 0.481816781597756
iteracion: 23, error: 0.44980012933770114
iteracion: 24, error: 0.17296322

In [222]:
x_train_sanos = np.array(balancear_datos("test/NORMAL"))
x_train_neum = np.array(balancear_datos("test/PNEUMONIA"))
w_train, b_train = gradiente_descendente(x_train_sanos, x_train_neum, 0.0001)


iteracion: 0, error: 0.25
iteracion: 1, error: 0.2359727059626393
iteracion: 2, error: 0.23432657470160534
iteracion: 3, error: 0.23313126714288573
iteracion: 4, error: 0.23197239612457174
iteracion: 5, error: 0.23082495162642863
iteracion: 6, error: 0.2296873391509958
iteracion: 7, error: 0.22855957129284
iteracion: 8, error: 0.2274417575120247
iteracion: 9, error: 0.22633400134659476
iteracion: 10, error: 0.2252363946548006
iteracion: 11, error: 0.22414901781250657
iteracion: 12, error: 0.22307194011662368
iteracion: 13, error: 0.22200522016056112
iteracion: 14, error: 0.2209489062040277
iteracion: 15, error: 0.21990303654657575
iteracion: 16, error: 0.21886763990607191
iteracion: 17, error: 0.21784273580109137
iteracion: 18, error: 0.21682833493571607
iteracion: 19, error: 0.21582443958522454
iteracion: 20, error: 0.21483104398120728
iteracion: 21, error: 0.21384813469477004
iteracion: 22, error: 0.21287569101656467
iteracion: 23, error: 0.21191368533252883
iteracion: 24, error: 0.2

Con escala 64 y alpha 0.001: 68 de 234 errores en sanos y 22 de 390 en enfermos -> 90 total
<br>Con escala 100 y alpha 0.001: 53 errores en sanos y 26 en enfermos -> 79 total
<br>Con escala 150 y alpha 0.001: 80 errores en sanos y 9 en enfermos  -> 89 total
<br>Con escala 200 y alpha 0.001: 49 errores en sanos y 15 en enfermos -> 64 total
<br>
<br>Con escala 250 y alpha 0.001: ya no encuentra bien el minimo
<br>si se hace con alpha 0.0001: 59 errores en sanos y 25 en enfermos


In [223]:
test_sanos = np.round(f(x_train_sanos, w_train, b_train))

suma_errores = 0
for i in range(len(test_sanos)):
    if test_sanos[i] == 1:
        suma_errores +=1

print(f'Diagnosticos a partir de descenso por gradiente: {test_sanos}; \nErrores: {suma_errores} de {len(test_sanos)}')

Diagnosticos a partir de descenso por gradiente: [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0.
 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 1. 0.
 0. 0. 1. 0. 1. 0. 1. 0. 1. 1. 1. 1. 0. 1. 0. 0. 1. 1. 0. 0. 0. 1. 0. 1.
 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 0. 1. 1. 0.
 0. 1. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0.
 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0.]; 
Errores: 59 de 234


In [224]:
test_neumonia = np.round(f(x_train_neum, w_train, b_train))


suma_errores = 0
for i in range(len(test_neumonia)):
    if test_neumonia[i] == 0:
        suma_errores +=1

print(f'Diagnosticos a partir de descenso por gradiente: {test_neumonia}; \nErrores: {suma_errores} de {len(test_neumonia)}')

Diagnosticos a partir de descenso por gradiente: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 0. 0. 1. 1. 1. 0. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1.
 1